In [32]:
import pandas as pd
import os
from io import StringIO
import sys; sys.path.append('../')

from lib.data import download, extract_zip
from lib.clean import (
    clean_column_names, clean_table, clean_beat, clean_category, float_to_int_str, clean_dates, clean_times,
    clean_races, clean_genders, float_to_int_str, clean_investigators, clean_finding
)

In [33]:
cr_zip_path = download('https://www.dropbox.com/s/392b045qtlsm9l9/2021-04-28_CR%20data%20to%20import.zip?dl=1')
cr_dir_path = extract_zip(cr_zip_path)
crids = ['20200001982', '20200003586', '1089903', '1092579']
officer_id_dict = {
    'encarnacion': '7939',
    'solano': '26969',
    'miles': '19051',
    'bedalow': '1652',  
    'astorga': '918',
    'khan': '14552'
}

In [34]:
cms_accused = pd.read_csv(os.path.join(cr_dir_path, '2021-04-28_CR data to import/CMS Spreadsheets - 2.03.2021/cms_accused.csv'))
cms_accused.columns
clear_accused = pd.read_csv(
    os.path.join(cr_dir_path, '2021-04-28_CR data to import/CLEAR Spreadsheets - 2.03.2021/clear_accused.csv'),
    encoding='latin-1'
).rename(columns={
    'ALLEGATION_CATEGORY_CD': 'ALLEGATION_CATEGORY_CODE',
    'FINDING_CD': 'FINAL_FINDING',
    'PENALTY_CD': 'FINAL_PENALTY',
})
accused = clean_table(pd.concat([cms_accused, clear_accused]))
accused = accused[accused.crid.isin(crids)]\
    .dropna(axis=1, how='all')\
    .drop(columns=[
        'middle_initial', 'first_name', 'race', 'gender', 'star_no', 'appointed_date', 'birth_year', 'unit_assigned_at_complaint',
        'current_unit_assigned', 'position_at_complaint', 'current_position', 'accused_on_duty', 'accused_arrested', 'allegation_category',
        'employee_on_duty', 'injury_condition', 'accusation_id', 'allegation_category_desc', 'no_of_days', 'final_penalty'
    ]).rename(columns={
        'last_name': 'officer_id',
    })\
    .pipe(clean_category)\
    .pipe(clean_finding)
accused = accused[accused.officer_id.notna()].reset_index(drop=True)
accused.loc[:, 'officer_id'] = accused.officer_id.str.lower().map(lambda x: officer_id_dict[x])
accused

oa_stmts = [
    (
        "INSERT INTO data_officerallegation ("
        "id, allegation_id, officer_id, final_finding, allegation_category_id, recc_finding, recc_outcome, "
        "final_outcome, final_outcome_class, created_at, updated_at) "
        "VALUES ((select max(id) + 1 from data_officerallegation), '%s', %s, '%s', %s, '', '', '', '', current_timestamp, current_timestamp);"
    ) % (
        row.crid, row.officer_id, row.final_finding, row.category_id
    ) for _, row in accused.iterrows()
]
print('\n'.join(oa_stmts))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
INSERT INTO data_officerallegation (id, allegation_id, officer_id, final_finding, allegation_category_id, recc_finding, recc_outcome, final_outcome, final_outcome_class, created_at, updated_at) VALUES ((select max(id) + 1 from data_officerallegation), '20200001982', 26969, '', 75, '', '', '', '', current_timestamp, current_timestamp);
INSERT INTO data_officerallegation (id, allegation_id, officer_id, final_finding, allegation_category_id, recc_finding, recc_outcome, final_outcome, final_outcome_class, created_at, updated_at) VALUES ((select max(id) + 1 from data_officerallegation), '20200001982', 7939, '', 75, '', '', '', '', current_timestamp, current_timestamp);
INSERT INTO data_officerallegation (id, allegation_id, officer_i

In [35]:
case_info = clean_table(pd.concat([
    pd.read_csv(os.path.join(cr_dir_path, '2021-04-28_CR data to import/CMS Spreadsheets - 2.03.2021/cms_case_info.csv')),
    pd.read_csv(
        os.path.join(cr_dir_path, '2021-04-28_CR data to import/CLEAR Spreadsheets - 2.03.2021/clear_case_info.csv'),
        encoding='latin-1'
    ).rename(columns={
        'CATEGORY_CODE': 'ALLEGATION_CATEGORY_CODE'
    }),
]))
case_info = case_info[case_info.crid.isin(crids)]\
    .dropna(axis=1, how='all')\
    .drop(columns=[
        'complaint_date', 'allegation_category', 'case_type', 'biased_language_i', 'pursuit_related_i',
        'domestic_violence_i', 'police_shooting', 'investigating_agency', 'current_status', 'days_to_closure',
        'category', 'bias_based_pro_i','alcohol_related_i','violence_at_work_i', 'closed_date'
    ]).rename(columns={
        'narrative': 'summary'
    })\
    .pipe(float_to_int_str, ['beat_of_incident'])\
    .pipe(clean_beat)\
    .pipe(clean_category)\
    .pipe(clean_dates, ['incident_date'])\
    .pipe(clean_times, ['incident_time'])
case_info.loc[:, 'incident_date'] = case_info.incident_date.str.cat(case_info.incident_time, sep=' ')
case_info.loc[case_info.complainant_type.str.lower() == 'civilian', 'is_officer_complaint'] = 'F'
case_info.loc[case_info.complainant_type.str.lower() == 'cpd employee', 'is_officer_complaint'] = 'T'
case_info.loc[:, 'location_code'] = case_info.location_code.fillna('').str.title()
case_info.loc[:, 'summary'] = case_info.summary.fillna('')
case_info = case_info.drop(columns=[
    'incident_time', 'state', 'beat_of_incident', 'allegation_category_code'
])
case_info

allegation_stmts = [
    (
        "INSERT INTO data_allegation ("
        "crid, incident_date, beat_id, location, most_common_category_id, is_officer_complaint, "
        "summary, add1, add2, city, source, subjects, created_at, updated_at, is_extracted_summary) "
        "VALUES ('%s', %s, %s, '%s', %s, '%s', '%s', '', '', 'CHICAGO ILLINOIS', '', '{}', current_timestamp, current_timestamp, 'F') "
        "ON CONFLICT DO NOTHING;"
    ) % (
        row.crid,
        'null' if pd.isnull(row.incident_date) else "'%s'" % row.incident_date,
        'null' if row.beat_id == '' else row.beat_id, row.location_code, row.category_id, row.is_officer_complaint, row.summary.replace("'", "''")
    )
    for _, row in case_info.drop_duplicates().iterrows()
]
print('\n'.join(allegation_stmts))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7,16,17,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
INSERT INTO data_allegation (crid, incident_date, beat_id, location, most_common_category_id, is_officer_complaint, summary, add1, add2, city, source, subjects, created_at, updated_at, is_extracted_summary) VALUES ('20200001982', null, 11, '', 75, 'F', 'CPD initiation report: The reporting subject alleges the accused officers searched his vehicle without justification and made fun of and antagonized him after accused stopped him for being illegally parked. The complainant got a ticket for being double parked and non-standing parking. ', '', '', 'CHICAGO ILLINOIS', '', '{}', current_timestamp, current_timestamp, 'F') ON CONFLICT DO NOTHING;
INSERT INTO data_allegation (crid, incident_date, beat_id, locatio

In [36]:
complainant = clean_table(pd.concat([
    pd.read_csv(os.path.join(cr_dir_path, '2021-04-28_CR data to import/CMS Spreadsheets - 2.03.2021/cms_complainant.csv')),
    pd.read_csv(
        os.path.join(cr_dir_path, '2021-04-28_CR data to import/CLEAR Spreadsheets - 2.03.2021/clear_complainant.csv'),
        encoding='latin-1'
    )
]))
complainant = complainant[complainant.crid.isin(crids)]\
    .dropna(axis=1, how='all')\
    .pipe(clean_races, ['race'])\
    .pipe(clean_genders, ['gender'])\
    .pipe(float_to_int_str, ['birth_year'])
complainant
com_stmts = [
    (
        "INSERT INTO data_complainant (id, gender, race, age, birth_year, created_at, updated_at, allegation_id) "
        "VALUES ((select max(id) + 1 from data_complainant), '%s', '%s', null, %s, current_timestamp, current_timestamp, '%s');"
    ) % (row.gender, row.race, row.birth_year if row.birth_year != '' else 'null', row.crid)
    for _, row in complainant.iterrows()
]
print('\n'.join(com_stmts))

INSERT INTO data_complainant (id, gender, race, age, birth_year, created_at, updated_at, allegation_id) VALUES ((select max(id) + 1 from data_complainant), 'M', 'Black', null, 1997, current_timestamp, current_timestamp, '20200001982');
INSERT INTO data_complainant (id, gender, race, age, birth_year, created_at, updated_at, allegation_id) VALUES ((select max(id) + 1 from data_complainant), 'M', 'White', null, 1978, current_timestamp, current_timestamp, '20200001982');
INSERT INTO data_complainant (id, gender, race, age, birth_year, created_at, updated_at, allegation_id) VALUES ((select max(id) + 1 from data_complainant), 'M', 'Asian/Pacific Islander', null, 1987, current_timestamp, current_timestamp, '20200003586');
INSERT INTO data_complainant (id, gender, race, age, birth_year, created_at, updated_at, allegation_id) VALUES ((select max(id) + 1 from data_complainant), '', '', null, 1966, current_timestamp, current_timestamp, '1089903');
INSERT INTO data_complainant (id, gender, race, a

In [37]:
pwit_stmts = [
    (
        "INSERT INTO data_policewitness (id, officer_id, created_at, updated_at, allegation_id)"
        "VALUES ((select max(id) + 1 from data_policewitness), %s, current_timestamp, current_timestamp, '%s');"
    ) % (officer_id, crid) for crid, officer_id in [('1092579', '14552')]
]
print('\n'.join(pwit_stmts))

INSERT INTO data_policewitness (id, officer_id, created_at, updated_at, allegation_id)VALUES ((select max(id) + 1 from data_policewitness), 14552, current_timestamp, current_timestamp, '1092579');


In [38]:
cms_investigator = clean_investigators(pd.concat([
    pd.read_csv(os.path.join(cr_dir_path, '2021-04-28_CR data to import/CMS Spreadsheets - 2.03.2021/cms_investigator.csv')),
    pd.read_csv(
        os.path.join(cr_dir_path, '2021-04-28_CR data to import/CLEAR Spreadsheets - 2.03.2021/clear_investigator.csv'),
        encoding='latin-1'
    ),
]))
cms_investigator = cms_investigator[cms_investigator.crid.isin(crids)]\
    .dropna(axis=1, how='all')
investigator_dict = {
    'Pfeifer': '1450',
    'Coleman': '1421',
    'Bowens': '1436',
    'Mack': '1452',
    'Kohs': '4788',
    'Nash': '4789',
    'Brown': '1297',
    'Klichowski': '1363',
    'Ward': '605',
    'Denham': '1051',
    'Tarver': '1418',
    'Brett': '1499',
}
cms_investigator.loc[:, 'investigator_id'] = cms_investigator.last_name.map(lambda x: investigator_dict.get(x, ''))
cms_investigator.loc[:, 'investigator_type'] = cms_investigator.investigator_type.fillna('investigator').str.title()
cms_investigator = cms_investigator.drop(columns=['assign_datetime']).drop_duplicates()
cms_investigator
iv_stmts = [
    (
        'INSERT INTO data_investigator '
        '(id, appointed_date, first_name, last_name, middle_initial, officer_id, suffix_name, gender, race, created_at, updated_at) '
        "VALUES (%s, '%s', '%s', '%s', '', null, '', '%s', '%s', current_timestamp, current_timestamp);"
    ) % (row.investigator_id, row.appointed_date, row.first_name, row.last_name, row.gender, row.race)
    for _, row in cms_investigator.loc[cms_investigator.investigator_id.isin(['4788', '4789'])].iterrows()
] + [
    (
        "INSERT INTO data_investigatorallegation "
        "(id, current_star, current_rank, current_unit_id, investigator_id, investigator_type, created_at, updated_at, allegation_id) "
        "VALUES ((select max(id) + 1 from data_investigatorallegation), %s, %s, %s, %s, '%s', current_timestamp, current_timestamp, '%s');"
    ) % (
        'null' if row.star_no == '' else "'%s'" % row.star_no,
        'null' if pd.isnull(row.current_position) else "'%s'" % row.current_position,
        'null' if row.unit_at_complaint == '' else row.unit_at_complaint,
        row.investigator_id, row.investigator_type, row.crid)
    for _, row in cms_investigator.drop(columns=['appointed_date']).drop_duplicates().iterrows()
]
print('\n'.join(iv_stmts))

../lib/clean.py:155: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.lower().fillna('').str.replace(r'^black.*', 'Black', regex=True)\
INSERT INTO data_investigator (id, appointed_date, first_name, last_name, middle_initial, officer_id, suffix_name, gender, race, created_at, updated_at) VALUES (4788, '2019-09-16', 'Madilyn', 'Kohs', '', null, '', 'F', 'White', current_timestamp, current_timestamp);
INSERT INTO data_investigator (id, appointed_date, first_name, last_name, middle_initial, officer_id, suffix_name, gender, race, created_at, updated_at) VALUES (4789, '2020-09-01', 'Valiza', 'Nash', '', null, '', 'F', 'Black', current_timestamp, current_timestamp);
INSERT INTO data_investigatorallegation (id, current_star, current_rank, current_unit_id, investigator_id, investigator_type, created_at, updated_at, allegation_id) VALUES ((select max(id) + 1 from data_investigatorallegation), null, 'SUPERVISING INV COPA

In [39]:
unit_history = pd.read_csv(StringIO("""FIRST_NME,MIDDLE_INITIAL,LAST_NME,APPOINTED_DATE,YOB,RACE_DESCR,SEX_CODE_CD,CPD_UNIT_ASSIGNED_NO,EFFECTIVE_DATE,END_DATE
EVAN,G,SOLANO,29-Jun-15,1991,WHITE HISPANIC,M,044,29-Jun-15,3-Dec-16
EVAN,G,SOLANO,29-Jun-15,1991,WHITE HISPANIC,M,016,4-Dec-16,
SAMMY, ,ENCARNACION,29-Jun-15,1989,WHITE HISPANIC,M,044,1-Jul-15,3-Dec-16
SAMMY, ,ENCARNACION,29-Jun-15,1989,WHITE HISPANIC,M,016,4-Dec-16,"""))\
    .pipe(clean_column_names)
for col in ['effective_date', 'end_date']:
    unit_history.loc[:, col] = pd.to_datetime(
        unit_history[col].str.replace(r'-(\d{2})$', r'-20\1', regex=True),
        format='%d-%b-%Y'
    ).dt.strftime("%Y-%m-%d").fillna("")
unit_history.loc[:, 'officer_id'] = unit_history.last_nme.str.lower().map(lambda x: officer_id_dict[x])
unit_dict = {
    '16': '17',
    '44': '30'
}
unit_history.loc[:, 'unit_id'] = unit_history.cpd_unit_assigned_no.astype(str).map(lambda x: unit_dict[x])
unit_history
oh_stmts = [
    "DELETE FROM data_officerhistory WHERE officer_id = %d;" % officer_id for officer_id in [26969, 7939]
] + [
    (
        "INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) "
        "VALUES ((SELECT max(id) + 1 FROM data_officerhistory), %s, %s, %s, %s, current_timestamp, current_timestamp);"
    ) % (
        'null' if r.effective_date == '' else "'%s'" % r.effective_date,
        'null' if r.end_date == '' else "'%s'" % r.end_date,
        r.officer_id, r.unit_id)
    for _, r in unit_history.iterrows()
]
print('\n'.join(oh_stmts))

DELETE FROM data_officerhistory WHERE officer_id = 26969;
DELETE FROM data_officerhistory WHERE officer_id = 7939;
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2015-06-29', '2016-12-03', 26969, 30, current_timestamp, current_timestamp);
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2016-12-04', null, 26969, 17, current_timestamp, current_timestamp);
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2015-07-01', '2016-12-03', 7939, 30, current_timestamp, current_timestamp);
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2016-12

In [40]:
insert_new_cat_stmt = (
    "INSERT INTO data_allegationcategory "
    "(id, category_code, category, allegation_name, on_duty, citizen_dept, created_at, updated_at) "
    "VALUES (417, 'S264', 'Illegal Search', 'Improper/Illegal Investigatory Stop', 'T', '', current_timestamp, current_timestamp);"
)
all_stmts = ['BEGIN;', insert_new_cat_stmt] + oh_stmts + allegation_stmts + oa_stmts + iv_stmts + pwit_stmts + com_stmts + ['COMMIT;']
print('\n'.join(all_stmts))

BEGIN;
INSERT INTO data_allegationcategory (id, category_code, category, allegation_name, on_duty, citizen_dept, created_at, updated_at) VALUES (417, 'S264', 'Illegal Search', 'Improper/Illegal Investigatory Stop', 'T', '', current_timestamp, current_timestamp);
DELETE FROM data_officerhistory WHERE officer_id = 26969;
DELETE FROM data_officerhistory WHERE officer_id = 7939;
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2015-06-29', '2016-12-03', 26969, 30, current_timestamp, current_timestamp);
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_officerhistory), '2016-12-04', null, 26969, 17, current_timestamp, current_timestamp);
INSERT INTO data_officerhistory (id, effective_date, end_date, officer_id, unit_id, created_at, updated_at) VALUES ((SELECT max(id) + 1 FROM data_of